## 8.3 Country Club case study

In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
# "sqlite_db_pythonsqlite.db" needs to be in working directory
pwd

'/Users/Carsten/OneDrive/Documents/Springboard/git_repositories/DataScienceSQLCountryClub'

In [4]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


In [7]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = "\
        SELECT name, membercost \
        FROM Facilities \
        WHERE membercost > 0;\
        "
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


In [8]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)

In [9]:
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', 5)
('Tennis Court 2', 5)
('Massage Room 1', 9.9)
('Massage Room 2', 9.9)
('Squash Court', 3.5)
